#Before we start 

This notebook was created using Google Colab, with Version 5.10.147+ of the platform and Python Version 3.9.16. Any additional packages installed in this notebook are compatible with these versions.

In this notebook, I attempted to run SQL in a Python environment. I recorded all of the SQL commands in this notebook and showcased the resulting tables as required by the tasks.

In [ ]:
pip install geopandas

In [24]:
import pandas as pd
import os

In [2]:
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database="haosheng_db", user='postgres', password='nycplanning2023', host='haoshengdb.cdx7p6lhdmdz.us-east-2.rds.amazonaws.com', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)

Connection established to:  ('PostgreSQL 14.6 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-12), 64-bit',)


#Task 5

Bullet Point 1 :Load the data/raw.csv into a Postgresql database name the table sample_311.

This SQL command creates a table called sample_311. After executing the command, I manually imported the CSV file into the table using PGAdmin User Interface.

In [ ]:
# CREATE TABLE sample_311 (
#     unique_key VARCHAR(20),
#     created_date TIMESTAMP,
#     agency VARCHAR(10),
#     agency_name VARCHAR(100),
#     complaint_type VARCHAR(50),
#     descriptor VARCHAR(100),
#     location_type VARCHAR(50),
#     incident_zip VARCHAR(10),
#     incident_address VARCHAR(100),
#     street_name VARCHAR(50),
#     address_type VARCHAR(20),
#     city VARCHAR(50),
#     status VARCHAR(20),
#     resolution_description VARCHAR(500),
#     resolution_action_updated_date TIMESTAMP,
#     community_board VARCHAR(20),
#     bbl VARCHAR(20),
#     borough VARCHAR(20),
#     x_coordinate_state_plane VARCHAR(20),
#     y_coordinate_state_plane VARCHAR(20),
#     open_data_channel_type VARCHAR(50),
#     park_facility_name VARCHAR(100),
#     park_borough VARCHAR(20),
#     latitude FLOAT,
#     longitude FLOAT,
#     location JSONB,
#     closed_date TIMESTAMP
# );

In [4]:
sample_311 = pd.read_sql("""
            select *
            from 
            sample_311
            """, con = conn)
sample_311

#Note the id and nta column were added by later tasks

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,closed_date,geom,id,nta
0,57207543,2023-04-02 15:50:05,HPD,Department of Housing Preservation and Develop...,WATER LEAK,SLOW LEAK,RESIDENTIAL BUILDING,10460,1511 COMMONWEALTH AVENUE,COMMONWEALTH AVENUE,...,PHONE,Unspecified,BRONX,40.838348,-73.869164,"{'latitude': '40.83834759380188', 'longitude':...",NaT,0101000020E6100000C2584962A07752C0990755F94E6B...,1,BX08
1,57192783,2023-03-31 18:58:07,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10312,58 BARB STREET,BARB STREET,...,PHONE,Unspecified,STATEN ISLAND,40.541851,-74.176489,"{'latitude': '40.54185136321202', 'longitude':...",NaT,0101000020E6100000B7F276974B8B52C0F224AE625B45...,99,SI32
2,57207922,2023-04-02 20:55:11,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10034,666 WEST 207 STREET,WEST 207 STREET,...,MOBILE,Unspecified,MANHATTAN,40.869224,-73.922202,"{'latitude': '40.86922407167794', 'longitude':...",NaT,0101000020E6100000DFA9915D057B52C0596000BC426F...,137,MN01
3,57212331,2023-04-02 17:54:19,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10460,1125 WYATT STREET,WYATT STREET,...,ONLINE,Unspecified,BRONX,40.839210,-73.876983,"{'latitude': '40.83920980381351', 'longitude':...",NaT,0101000020E61000001C6B927D207852C049EE123A6B6B...,287,BX17
4,57209274,2023-04-02 16:31:16,HPD,Department of Housing Preservation and Develop...,SAFETY,SMOKE DETECTOR,RESIDENTIAL BUILDING,10456,3462 3 AVENUE,3 AVENUE,...,ONLINE,Unspecified,BRONX,40.829861,-73.906374,"{'latitude': '40.829861467032806', 'longitude'...",NaT,0101000020E610000090ACEC09027A52C0E98E8AE6386A...,390,BX35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10225,57162398,2023-03-28 06:37:12,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10468,2710 MORRIS AVENUE,MORRIS AVENUE,...,ONLINE,Unspecified,BRONX,40.867919,-73.895604,"{'latitude': '40.8679185556592', 'longitude': ...",2023-03-29 16:15:17,0101000020E61000006057D494517952C0EC128AF4176F...,10226,BX05
10226,57164260,2023-03-28 06:36:22,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11103,30-08 42 STREET,42 STREET,...,ONLINE,Unspecified,QUEENS,40.762720,-73.913750,"{'latitude': '40.76272015085797', 'longitude':...",2023-03-29 18:30:52,0101000020E610000092FCCBE07A7A52C0B4F75BD0A061...,10227,QN70
10227,57167417,2023-03-28 06:33:20,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10037,2225 5 AVENUE,5 AVENUE,...,ONLINE,Unspecified,MANHATTAN,40.813057,-73.937387,"{'latitude': '40.81305735256445', 'longitude':...",2023-03-28 20:33:46,0101000020E61000007A612525FE7B52C0B18469431268...,10228,MN34
10228,57164717,2023-03-28 06:32:58,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10458,2601 BAINBRIDGE AVENUE,BAINBRIDGE AVENUE,...,ONLINE,Unspecified,BRONX,40.864180,-73.892378,"{'latitude': '40.86418005715264', 'longitude':...",2023-03-29 15:56:36,0101000020E61000008B7FE8B81C7952C0BAA9BD739D6E...,10229,BX05


Bullet Point 2 :Perform aggregation in SQL and store the results in a table

The following command creates a time series table that aggregates the count of service requests by type and time.

In [ ]:
# CREATE TABLE time_series_raw AS
# SELECT
#   date_trunc('hour', created_date) AS created_date_hour,
#   complaint_type,
#   COUNT(*) AS count
# FROM
#   sample_311
# GROUP BY
#   created_date_hour,
#   complaint_type;

In [5]:
time_series_raw = pd.read_sql("""
            select *
            from 
            time_series_raw
            """, con = conn)
time_series_raw

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,created_date_hour,complaint_type,count
0,2023-04-01 18:00:00,PAINT/PLASTER,4
1,2023-03-29 01:00:00,HEAT/HOT WATER,16
2,2023-03-30 22:00:00,ELECTRIC,4
3,2023-03-30 09:00:00,DOOR/WINDOW,9
4,2023-04-02 22:00:00,WATER LEAK,1
...,...,...,...
1219,2023-03-28 20:00:00,PLUMBING,6
1220,2023-04-02 14:00:00,ELECTRIC,2
1221,2023-04-01 13:00:00,UNSANITARY CONDITION,16
1222,2023-03-30 02:00:00,UNSANITARY CONDITION,6


#Task 6

Bullet Point 1 :Load the NTA data to the database as a spatial table

After executing the command, I manually imported the CSV file into the table using PGAdmin User Inferface.

In [ ]:
# CREATE TABLE NTA (
#     x FLOAT,
#     city VARCHAR(50),
#     name VARCHAR(255),
#     activated TIMESTAMP,
#     ntacode VARCHAR(10),
#     location VARCHAR(255),
#     provider VARCHAR(255),
#     coundist VARCHAR(10),
#     y FLOAT,
#     ntaname VARCHAR(255),
#     location_t VARCHAR(50),
#     ssid VARCHAR(255),
#     postcode VARCHAR(10),
#     bbl VARCHAR(20),
#     borocd VARCHAR(10),
#     objectid VARCHAR(10),
#     boroname VARCHAR(50),
#     sourceid VARCHAR(20),
#     boroct2010 VARCHAR(10),
#     boro VARCHAR(2),
#     lon FLOAT,
#     remarks TEXT,
#     ct2010 VARCHAR(10),
#     borocode VARCHAR(10),
#     bin VARCHAR(20),
#     type VARCHAR(20),
#     doitt_id VARCHAR(10),
#     lat FLOAT,
#     geom GEOMETRY(Point, 4326)
# );

Bullet Point 2 :Spatial join in SQL between sample_311 and the NTA table and add a nta column to sample_311

First install PostGIS

In [ ]:
# CREATE EXTENSION postgis;

Create an empty column nta

In [ ]:
# ALTER TABLE sample_311
# ADD COLUMN nta varchar(50);

Add a primary key to sample_311 table before join

In [ ]:
# ALTER TABLE sample_311 ADD COLUMN id serial PRIMARY KEY;

Spatial join based on shortest distance, then update the sample_311 empty column 

In [ ]:
# UPDATE sample_311
# SET nta = nta_table.ntacode
# FROM (
#   SELECT
#     sample_311.id, nta.ntacode,
#     ST_Distance(sample_311.geom, nta.geom) AS distance
#   FROM
#     sample_311
#     CROSS JOIN LATERAL (
#       SELECT *
#       FROM nta
#       ORDER BY sample_311.geom <-> nta.geom ASC
#       LIMIT 1
#     ) AS nta
# ) AS nta_table
# WHERE sample_311.id = nta_table.id;

In [6]:
sample_311 = pd.read_sql("""
            select *
            from 
            sample_311
            """, con = conn)
sample_311

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,closed_date,geom,id,nta
0,57207543,2023-04-02 15:50:05,HPD,Department of Housing Preservation and Develop...,WATER LEAK,SLOW LEAK,RESIDENTIAL BUILDING,10460,1511 COMMONWEALTH AVENUE,COMMONWEALTH AVENUE,...,PHONE,Unspecified,BRONX,40.838348,-73.869164,"{'latitude': '40.83834759380188', 'longitude':...",NaT,0101000020E6100000C2584962A07752C0990755F94E6B...,1,BX08
1,57192783,2023-03-31 18:58:07,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10312,58 BARB STREET,BARB STREET,...,PHONE,Unspecified,STATEN ISLAND,40.541851,-74.176489,"{'latitude': '40.54185136321202', 'longitude':...",NaT,0101000020E6100000B7F276974B8B52C0F224AE625B45...,99,SI32
2,57207922,2023-04-02 20:55:11,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10034,666 WEST 207 STREET,WEST 207 STREET,...,MOBILE,Unspecified,MANHATTAN,40.869224,-73.922202,"{'latitude': '40.86922407167794', 'longitude':...",NaT,0101000020E6100000DFA9915D057B52C0596000BC426F...,137,MN01
3,57212331,2023-04-02 17:54:19,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10460,1125 WYATT STREET,WYATT STREET,...,ONLINE,Unspecified,BRONX,40.839210,-73.876983,"{'latitude': '40.83920980381351', 'longitude':...",NaT,0101000020E61000001C6B927D207852C049EE123A6B6B...,287,BX17
4,57209274,2023-04-02 16:31:16,HPD,Department of Housing Preservation and Develop...,SAFETY,SMOKE DETECTOR,RESIDENTIAL BUILDING,10456,3462 3 AVENUE,3 AVENUE,...,ONLINE,Unspecified,BRONX,40.829861,-73.906374,"{'latitude': '40.829861467032806', 'longitude'...",NaT,0101000020E610000090ACEC09027A52C0E98E8AE6386A...,390,BX35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10225,57162398,2023-03-28 06:37:12,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10468,2710 MORRIS AVENUE,MORRIS AVENUE,...,ONLINE,Unspecified,BRONX,40.867919,-73.895604,"{'latitude': '40.8679185556592', 'longitude': ...",2023-03-29 16:15:17,0101000020E61000006057D494517952C0EC128AF4176F...,10226,BX05
10226,57164260,2023-03-28 06:36:22,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11103,30-08 42 STREET,42 STREET,...,ONLINE,Unspecified,QUEENS,40.762720,-73.913750,"{'latitude': '40.76272015085797', 'longitude':...",2023-03-29 18:30:52,0101000020E610000092FCCBE07A7A52C0B4F75BD0A061...,10227,QN70
10227,57167417,2023-03-28 06:33:20,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10037,2225 5 AVENUE,5 AVENUE,...,ONLINE,Unspecified,MANHATTAN,40.813057,-73.937387,"{'latitude': '40.81305735256445', 'longitude':...",2023-03-28 20:33:46,0101000020E61000007A612525FE7B52C0B18469431268...,10228,MN34
10228,57164717,2023-03-28 06:32:58,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10458,2601 BAINBRIDGE AVENUE,BAINBRIDGE AVENUE,...,ONLINE,Unspecified,BRONX,40.864180,-73.892378,"{'latitude': '40.86418005715264', 'longitude':...",2023-03-29 15:56:36,0101000020E61000008B7FE8B81C7952C0BAA9BD739D6E...,10229,BX05


Bullet Point 3 :Perform the same aggregation in Task 4 and store the result in a table.

In [ ]:
# CREATE TABLE recent_appliance_complaints AS
# SELECT nta,
#        COUNT(*) AS count,
#        nta.geom AS geom
# FROM (
#   SELECT unique_key, 
#          complaint_type, 
#          nta, 
#          DATE_TRUNC('day', created_date) AS date_trunc
#   FROM sample_311
#   WHERE complaint_type = 'APPLIANCE'
#     AND created_date >= CURRENT_DATE - INTERVAL '7 day'
# ) AS recent_complaints
# JOIN nta ON recent_complaints.nta = nta.ntacode
# GROUP BY nta, nta.geom;

In [16]:
recent_appliance_complaints = pd.read_sql("""
            select *
            from 
            recent_appliance_complaints
            """, con = conn)
recent_appliance_complaints

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,nta,count,geom
0,MN19,1,0101000020E610000088C9ECD3167E52C075BC1A4DCF60...
1,BK79,3,0101000020E610000070B56728EE7A52C09BD8A8A3E356...
2,MN17,1,0101000020E6100000EFA1A2923A7F52C0F28DBBCFF15F...
3,MN12,2,0101000020E6100000889446F9F07D52C0049C143D7066...
4,QN17,4,0101000020E6100000909B67B7967552C0C5002C45F25B...
...,...,...,...
1987,BK90,1,0101000020E6100000BDA1E1E6547C52C014A79355115A...
1988,MN33,1,0101000020E610000095F0A9A0A27C52C072104EEFE265...
1989,QN17,4,0101000020E6100000B6961F93127652C05C1444F55A5C...
1990,QN54,1,0101000020E610000042E8A108A97452C078CF7A48F858...


In [19]:
# Parse EWKB string into geometry
from shapely import wkb
from binascii import unhexlify

def parse_ewkb_string(ewkb_str):
    ewkb_bytes = unhexlify(ewkb_str)
    return wkb.loads(ewkb_bytes)

recent_appliance_complaints['geom'] = recent_appliance_complaints['geom'].apply(parse_ewkb_string)

Bonus: Export the table with NTA geometry and complaint count into a shapefile under the data folder.

In [25]:
import geopandas as gpd

gdf = gpd.GeoDataFrame(recent_appliance_complaints, geometry='geom')

if not os.path.exists("data"):
    os.makedirs("data")

gdf.to_file('data/shapefile.shp', driver='ESRI Shapefile')